## Algoritmo para detectar hexágonos em imagem:

1. Carregar a imagem
2. Redimensionar a imagem, aplicando-se um fator de redução
3. Converter a imagem para escala de cinza
4. Introduzir _blur_ gaussiano para reduzir ruídos na imagem
5. Fazer segmentação da imagem utilizando _thresholding_
6. Fazer aproximação de contorno (_contour approximation_) utilizando o algoritmo de __Ramer–Douglas–Peucker__
7. Identificar as figuras com contorno aproximado por exatamente 6 segmentos
8. Para as figuras identificadas no passo __7__, calcular os momentos das imagens (_image moments_) e obter os perímetros e as áreas dos seus respectivos contornos
9. Com os perímetros obtidos no passo __8__, calcular $\frac{3\sqrt{3}}{2}(\frac{\text{perímetro}}{6})^2$ (correspondente à área do hexágono)
10. Utilizando os resultdos dos passos __8__ e __9__, calcular a razão $r = (\text{área do contorno}) / (\text{área do hexágono})$
11. Classificar como hexágono as figuras que verificarem as condições seguintes:
    * possuir contorno aproximado com exatamente 6 segmentos
    * possuir valor da razão $r$ tal que $0.90 \leq r \leq 1.10$ 

In [ ]:
import imutils
import cv2
import math

In [51]:
class Hexagon():
  def find(self, contour):
    arc = cv2.arcLength(contour, True)
    polyg = cv2.approxPolyDP(contour, 0.04 * arc, True)

    area = cv2.contourArea(contour)
    area_hex = ((arc / 6)**2) * 3 * math.sqrt(3) / 2
    r = area / area_hex

    if len(polyg) == 6:
     print('Hexagono dectectado -> l=', len(polyg), ' r=', r)

    hex = True if (len(polyg) == 6) and (r >= 0.9) and (r <= 1.1) else False

    return hex, arc

In [52]:
im = cv2.imread('./poligonos.png')

im_res = imutils.resize(im, width=300)
ratio = im.shape[0] / float(im_res.shape[0])

im_grey = cv2.cvtColor(im_res, cv2.COLOR_BGR2GRAY)
im_grey_blur = cv2.GaussianBlur(im_grey, (5, 5), 0)
_, im_grey_blur_thr = cv2.threshold(im_grey_blur, 200, 255, cv2.THRESH_BINARY_INV)

In [53]:
contours = cv2.findContours(im_grey_blur_thr.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)

In [54]:
hexagon = Hexagon()

for i in contours:
  is_hex, arc_hex = hexagon.find(i)

  if is_hex:
    mu = cv2.moments(i)
    x = int((mu['m10'] / mu['m00']) * ratio)
    y = int((mu['m01'] / mu['m00']) * ratio)

    i = i.astype('float')
    i *= ratio
    i = i.astype('int')

    cv2.drawContours(im, [i], -1, (0, 0, 255), 2)
    text = 'arc=%d' % round(arc_hex * ratio, 2)
    cv2.putText(im, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.imshow('Im', im)
    cv2.waitKey(0)

Hexagono dectectado -> l= 6  r= 0.9174674013579696


In [55]:
cv2.destroyAllWindows()